#### Автор: Сергеев Константин Олегович (ММОВС23)

# RoBERTa: Улучшенная модель для обработки естественного языка

## Введение

В данной работе рассматривается модель RoBERTa, представленная в 2019 году группой исследователей из Facebook AI и до сих пор пользующаяся большой популярностью.

RoBERTa является улучшенной версией модели BERT, разработанной для решения задач обработки естественного языка. Основные улучшения достигаются за счёт изменения в процессе предобучения, увеличения объема данных и оптимизации гиперпараметров.

Модель RoBERTa показала значительные улучшения в различных задачах, таких как классификация текста, вопросно-ответные системы и понимание текста.

В данной работе выдвигаются гипотезы по замене болоков построения модели на более современные и эффективные.

В частности реализовывается замена Positional Encoding части с базового PE на RoPE, для улучшения качества при работе с длинными последовательностями.

## Архитектура

Модель RoBERTa основана на архитектуре трансформера, которая включает в себя несколько слоев self-attention и полносвязных слоев.

Для токенизации используется метод BPE с размером словаря в 50K токенов. Авторы выбрали BPE из-за его эффективности обрабатывать большие и разнообразные корпуса текстов, избегая проблем с неизвестными токенами.

Positional Encoding в RoBERTa реализован с использованием традиционных синусоидальных и косинусоидальных функций.

Модель RoBERTa имеет два варианта: RoBERTa-BASE с 12 слоями и 110 миллионами параметров, и RoBERTa-LARGE с 24 слоями и 355 миллионами параметров.

## Обучение

Специфика обучения RoBERTa заключалась в нескольких ключевых изменения процесса обучения BERT:

- Модель обучалась на значительно большем объеме данных (160 ГБ текста), включая оригинальный корпус BERT (16 ГБ) и дополнительные наборы данных: CC-NEWS, OpenWebText и STORIES. Авторы аргументировали это тем, что увеличение объема и разнообразия данных улучшает производительность модели.
- RoBERTa использовала только задачу маскированного языкового моделирования (MLM), отказавшись от задачи предсказания следующего предложения (NSP), так как эксперименты показали, что NSP не улучшает производительность.
- Обучение проводилось с большими батчами (8192 последовательности) и динамическим маскированием, что, по мнению авторов, позволяет модели лучше обобщать.

Использовался оптимизатор Adam с параметрами β1 = 0.9, β2 = 0.98, ε = 1e-6 и линейным затуханием скорости обучения. Модель обучалась в течение 500 000 шагов, что значительно дольше, чем оригинальный BERT.

## Метрики

Для оценки производительности RoBERTa использовались несколько бенчмарков области NLP:

- Основным был GLUE (General Language Understanding Evaluation), включающий 9 различных задач понимания естественного языка. RoBERTa достигла state-of-the-art результатов на задачах GLUE и показала наивысший средний балл на момент публикации.
- Также модель оценивалась на наборе данных SQuAD (Stanford Question Answering Dataset). На SQuAD 2.0 RoBERTa превзошла предыдущий state-of-the-art результат, улучшив показатели EM (Exact Match) на 0.4 пункта и F1 на 0.6 пункта по сравнению с XLNet.
- На RACE (Reading Comprehension from Examinations) RoBERTa также достигла наилучших результатов, превзойдя XLNet на 1.5 пункта в общей точности.

## Выдвижение гипотез
Гипотезы, которые можно проверить, после анализа статьи:
1. Замена текущего позиционного кодирования на более современный RoPE (Rotary Position Embedding) могло бы повысить производительность модели при работе с длинными последовательностями.
2. Увеличение размера словаря BPE до 100K токенов может улучшить способность модели обрабатывать редкие слова и специфические термины.
3. Использование модификаций оптимизатора Adam, например AdamW, может улучшить сходимость и качество модели.

## Проверка гипотезы
Проверим гипотезу 1, для этого:

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3" # 4 H100

In [2]:
import warnings
warnings.filterwarnings("ignore")

- Скачаем оригинальную модель FacebookAI/roberta-base

In [3]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification


model_name = "FacebookAI/roberta-base"
num_labels = 2

tokenizer = RobertaTokenizer.from_pretrained(model_name)
model_base = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


- Скачаем и токенизируем учебный набор данных с задачей бинарной классификации на сентимент отзывов о фильмах с сайта IMDB, при этом возьмём примеры только с максимальной длиной поддерживаемой моделью - 512

In [4]:
#!pip install datasets

In [5]:
from datasets import load_dataset


# Функция для токенизации датасета
def tokenize_function(examples):
    return tokenizer(examples["text"],
                     truncation=True, padding="max_length", max_length=512)

# Загрузка датасета
dataset = load_dataset("imdb")["train"].train_test_split(test_size=0.1, seed=42)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [6]:
# Пример текста
tokenized_datasets['test'][0]['text'], tokenized_datasets['test'][0]['label']

('There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...',
 1)

In [7]:
def filter_long_examples(example):
    return sum(example['attention_mask']) == 512

filtered_datasets = tokenized_datasets.filter(filter_long_examples)
filtered_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3104
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 321
    })
})

In [8]:
import pandas as pd


# Распределение классов
pd.Series(filtered_datasets['test']['label']).value_counts()

1    179
0    142
Name: count, dtype: int64

- Дообучим базовую модель и замерим точность

In [9]:
from transformers import Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.metrics import accuracy_score, f1_score
import numpy as np


# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    logging_steps=5,
    num_train_epochs=100,
    save_steps=100000000
)

# Функция для вычисления метрик
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')
    return {"accuracy": accuracy, "f1": f1}

# Создание тренера
trainer = Trainer(
    model=model_base,
    args=training_args,
    train_dataset=filtered_datasets["train"],
    eval_dataset=filtered_datasets["test"],
    compute_metrics=compute_metrics,
)

# Обучение модели
trainer.train()

# Оценка модели
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.685300,0.672868,0.557632,0.716000
2,0.569700,0.372995,0.890966,0.905149
3,0.325500,0.287347,0.903427,0.912181
4,0.292400,0.268291,0.890966,0.906166
5,0.227600,0.234794,0.912773,0.923497
6,0.206900,0.216988,0.925234,0.931429
7,0.183400,0.200830,0.915888,0.925208
8,0.155300,0.199137,0.928349,0.935574
9,0.137900,0.199096,0.931464,0.937143
10,0.108100,0.217616,0.928349,0.938005


{'eval_loss': 0.5357391834259033,
 'eval_accuracy': 0.9314641744548287,
 'eval_f1': 0.9388888888888889,
 'eval_runtime': 0.429,
 'eval_samples_per_second': 748.194,
 'eval_steps_per_second': 4.662,
 'epoch': 100.0}

- Заменим стандартный Positional Encoding на RoPE, применим RoPE ко всем слоям модели, как и предполагает правильная реализация RoPE

In [10]:
import math
import torch
import torch.nn as nn
from transformers import RobertaModel, RobertaConfig
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers.models.roberta.modeling_roberta import (RobertaEmbeddings,
                                                          RobertaSelfAttention,
                                                          RobertaAttention,
                                                          RobertaLayer,
                                                          RobertaEncoder)

# Функции для RoPE
def rotate_every_two(x):
    # Функция для поворота элементов вектора
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    x = torch.stack((-x2, x1), dim=-1)
    return x.flatten(-2)

def apply_rotary_pos_emb(q, k, sin, cos):
    # Применение RoPE к вектору запросов и ключей
    sin = sin.unsqueeze(0).unsqueeze(0)  # Добавляем размерности для совместимости
    cos = cos.unsqueeze(0).unsqueeze(0)
    q_embed = (q * cos) + (rotate_every_two(q) * sin)
    k_embed = (k * cos) + (rotate_every_two(k) * sin)
    return q_embed, k_embed

class RotaryPositionEmbedding(nn.Module):
    # Класс для создания синусоидальных и косинусоидальных эмбеддингов
    def __init__(self, dim, max_position_embeddings):
        super().__init__()
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq)
        self.max_seq_len = max_position_embeddings

    def forward(self, seq_len):
        # Генерация синусоидальных и косинусоидальных эмбеддингов
        t = torch.arange(seq_len, device=self.inv_freq.device).type_as(self.inv_freq)
        freqs = torch.einsum("i,j->ij", t, self.inv_freq)
        emb = torch.cat((freqs, freqs), dim=-1)
        return emb.sin(), emb.cos()

# Переопределяем RobertaEmbeddings, чтобы убрать позиционные эмбеддинги
class CustomRobertaEmbeddings(RobertaEmbeddings):
    def forward(self, input_ids=None, token_type_ids=None, position_ids=None, inputs_embeds=None, past_key_values_length=0):
        # Все то же самое, но без добавления позиционных эмбеддингов
        if input_ids is not None:
            input_shape = input_ids.size()
        else:
            input_shape = inputs_embeds.size()[:-1]

        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=self.position_ids.device)

        if inputs_embeds is None:
            inputs_embeds = self.word_embeddings(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        # Не добавляем позиционные эмбеддинги
        embeddings = inputs_embeds + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

# Переопределяем RobertaSelfAttention для применения RoPE
class CustomRobertaSelfAttention(RobertaSelfAttention):
    def __init__(self, config):
        super().__init__(config)
        self.rotary_emb = RotaryPositionEmbedding(
            config.hidden_size // config.num_attention_heads,
            config.max_position_embeddings)

    def forward(self,
                hidden_states,
                attention_mask=None,
                head_mask=None,
                encoder_hidden_states=None,
                encoder_attention_mask=None,
                past_key_value=None,
                output_attentions=False):

        # Выполняем стандартные операции получения q, k, v
        mixed_query_layer = self.query(hidden_states)
        key_layer = self.key(hidden_states)
        value_layer = self.value(hidden_states)

        # Изменяем форму для многоголового внимания
        batch_size, seq_length, _ = hidden_states.size()
        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(key_layer)
        value_layer = self.transpose_for_scores(value_layer)

        # Применяем RoPE к q и k
        sin, cos = self.rotary_emb(seq_length)
        query_layer, key_layer = apply_rotary_pos_emb(query_layer, key_layer, sin, cos)

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)

        if attention_mask is not None:
            attention_scores = attention_scores + attention_mask

        attention_probs = nn.functional.softmax(attention_scores, dim=-1)
        attention_probs = self.dropout(attention_probs)

        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        return outputs

# Создаем модель с правильной реализацией RoPE
class Model_RoPE_Layers(nn.Module):
    def __init__(self, model_name, num_labels):
        super(Model_RoPE_Layers, self).__init__()

        # Инициализируем модель
        self.config = RobertaConfig.from_pretrained(model_name, num_labels=num_labels)
        self.model = RobertaModel(self.config)

        # Заменяем embeddings на наши
        self.model.embeddings = CustomRobertaEmbeddings(self.config)

        # Заменяем все слои внимания на CustomRobertaSelfAttention
        for layer in self.model.encoder.layer:
            layer.attention.self = CustomRobertaSelfAttention(self.config)

        self.classifier = nn.Linear(self.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None):
        # Получаем выходы модели
        outputs = self.model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs[0]

        # Используем представление [CLS] токена для классификации
        logits = self.classifier(sequence_output[:, 0, :])

        # Вычисляем ошибку если есть labels
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.config.num_labels), labels.view(-1))

        return SequenceClassifierOutput(loss=loss, logits=logits)

model_rope = Model_RoPE_Layers(model_name, num_labels)

- Также дообучим новую модель и замерим точность

In [11]:
# Параметры обучения
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    logging_steps=5,
    num_train_epochs=100,
    save_steps=100000000,
)

# Создание тренера
trainer = Trainer(
    model=model_rope,
    args=training_args,
    train_dataset=filtered_datasets["train"],
    eval_dataset=filtered_datasets["test"],
    compute_metrics=compute_metrics,
)

# Обучение модели
trainer.train()

# Оценка модели
trainer.evaluate()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.741300,0.685950,0.557632,0.716000
2,0.712000,0.682900,0.557632,0.716000
3,0.697300,0.682504,0.557632,0.716000
4,0.685400,0.673074,0.557632,0.716000
5,0.696700,0.692828,0.557632,0.716000
6,0.699600,0.721025,0.442368,0.000000
7,0.680200,0.687571,0.557632,0.716000
8,0.711800,0.698232,0.448598,0.022099
9,0.674400,0.660967,0.591900,0.478088
10,0.664700,0.622180,0.688474,0.691358


{'eval_loss': 0.4826228618621826,
 'eval_accuracy': 0.7819314641744548,
 'eval_f1': 0.8044692737430168,
 'eval_runtime': 0.4725,
 'eval_samples_per_second': 679.343,
 'eval_steps_per_second': 4.233,
 'epoch': 100.0}

## Результат

- Модель с собственной реализацией RoPE: инициализируется, градиенты текут, train_loss падает и f1 растёт
- Модель с добавлением собственной реализации RoPE справилась с задачей бинарной классификации длинных текстов хуже, чем базовая модель RoBERTa при одинаковых прочих вводных: 0.8 f1 против 0.94 f1 соответсвенно.
- Скорее всего так произошло из-за потерянной предобученности модели и возможно неточности в собственной реализации RoPE, что следовало ожидать.


## Идеи развития
- Проверить собственную реализацию RoPE
- Предворительно инициализировать обе модели случайными весами, чтобы нивелировать преимущество предобученности
- Обучать дольше и на большом количестве данных
- Подобрать оптимальные гиперпараметры для обоих моделей
- Проверить гипотезы 2 и 3


## Ссылки
- Оригинальная статья RoBERTa 2019г. - https://arxiv.org/pdf/1907.11692
- Карточка на HF оригинальной модели RoBERTa - https://huggingface.co/FacebookAI/roberta-base
- Оригинальная статья RoPE 2023 г. - https://arxiv.org/abs/2104.09864